# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [8]:
import pandas as pd

url="https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

df=pd.read_csv(url)
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [11]:
df1k=df[df["Total Claim Amount"]>1000]
df1k

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
30,30,GX19342,California,8491.465719,No,Premium,High School or Below,1/28/11,Unemployed,F,...,0.0,3,Personal Auto,Personal L3,Offer1,Branch,1193.115879,Four-Door Car,Large,A
55,55,XQ74827,Arizona,19435.629240,No,Extended,High School or Below,2/1/11,Employed,F,...,0.0,3,Special Auto,Special L3,Offer1,Agent,2294.631639,Luxury Car,Medsize,A
74,74,LD75023,Oregon,26346.085820,No,Extended,High School or Below,1/20/11,Employed,M,...,0.0,2,Personal Auto,Personal L2,Offer1,Web,1605.600000,Luxury SUV,Medsize,NaN
77,77,YC54142,Washington,74228.516040,No,Extended,High School or Below,1/26/11,Unemployed,M,...,0.0,2,Personal Auto,Personal L1,Offer1,Branch,1742.400000,Luxury Car,Medsize,NaN
99,99,JY66887,California,16320.767160,No,Extended,High School or Below,2/27/11,Unemployed,F,...,3.0,5,Personal Auto,Personal L2,Offer2,Branch,1778.400000,Luxury Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10874,10874,PD52748,NaN,10036.536340,NaN,Extended,High School or Below,2/12/11,Unemployed,F,...,1.0,8,Personal Auto,Personal L3,Offer1,Call Center,1353.289060,SUV,Medsize,NaN
10879,10879,FK95213,California,18611.453560,No,Extended,Master,1/16/11,Employed,F,...,0.0,7,Personal Auto,Personal L3,Offer2,Web,1021.654003,NaN,NaN,NaN
10900,10900,VN73653,Oregon,6423.739052,No,Basic,College,2/13/11,Unemployed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer3,Agent,1621.560425,Luxury SUV,Medsize,NaN
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A


In [16]:
dfyes=df[df["Response"]=="Yes"]
dfyes_pivot=dfyes.pivot_table(index="Policy Type", columns="Gender", values="Total Claim Amount", aggfunc="mean")
dfyes_pivot


Gender,F,M
Policy Type,,
Corporate Auto,433.738499,408.582459
Personal Auto,452.965929,457.010178
Special Auto,453.280164,429.527942


In [21]:
#number of customers in states:

dfstate=df.groupby("State").size()
dfstate

#Every state seems to have more than 500 customers



,0
State,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


In [31]:
df_pivot_gen_edu_mean=df.pivot_table(index="Gender", columns="Education", values="Customer Lifetime Value", aggfunc="mean")
print("AVERAGE")
print(df_pivot_gen_edu_mean)

df_pivot_gen_edu_max=df.pivot_table(index="Gender", columns="Education", values="Customer Lifetime Value", aggfunc="max")
print("MAXIMUM")
print(df_pivot_gen_edu_max)

df_pivot_gen_edu_min=df.pivot_table(index="Gender", columns="Education", values="Customer Lifetime Value", aggfunc="min")
print("MINIMUM")
print( df_pivot_gen_edu_min)

AVERAGE
Education     Bachelor      College       Doctor  High School or Below  \
Gender                                                                   
F          7874.269478  7748.823325  7328.508916           8675.220201   
M          7703.601675  8052.459288  7415.333638           8149.687783   

Education       Master  
Gender                  
F          8157.053154  
M          8168.832659  
MAXIMUM
Education     Bachelor      College       Doctor  High School or Below  \
Gender                                                                   
F          73225.95652  61850.18803  44856.11397           55277.44589   
M          67907.27050  61134.68307  32677.34284           83325.38119   

Education       Master  
Gender                  
F          51016.06704  
M          50568.25912  
MINIMUM
Education     Bachelor      College       Doctor  High School or Below  \
Gender                                                                   
F          1904.000852  1898.68368

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here